---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import os
os.chdir('D:\Classes\[Coursera] Introduction to Data Science in Python')

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  ) 
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    univ_towns_df = pd.DataFrame(columns = ['State', 'RegionName'])
    univ_towns_raw = pd.read_csv('university_towns.txt', sep='\n', header=None, names=['raw'])

    current_state = None

    for row in univ_towns_raw.itertuples():
        if '[ed' in row[1]:
            current_state = row[1].split('[')[0]
        else:
            if '(' in row[1]:
                current_region = row[1].split(' (')[0]
            else:
                current_region = row[1]
            univ_towns_df = univ_towns_df.append(pd.Series({'State': current_state, 'RegionName': current_region}),ignore_index=True)
            
    return univ_towns_df

In [4]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    
    q1 < q2 > q3 > q4 > q5 < q6 < q7 < q8. Then, the recession start is q3, bottom is q5 and end is q7.
    '''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    gdp.columns = ['year_q', 'GDP']
    
    def split_year(row):
        return int(row.split('q')[0]) >= 2000
    
    gdp = gdp[gdp['year_q'].apply(split_year)]       #return only year 2000 above
    
    start_q = []                                     #list of recession starts quarter            
    
    for i in range(2,len(gdp)-2):
        last_2 = gdp.iloc[i-2]['GDP']
        last_ = gdp.iloc[i-1]['GDP']
        current_ = gdp.iloc[i]['GDP']
        next_ = gdp.iloc[i+1]['GDP']
        next_2 = gdp.iloc[i+2]['GDP']
        if last_2 > last_ and current_ < last_ and current_ < next_  and next_ < next_2:      #recession bottom
            for j in range(0,len(gdp)-2):                                                     #check start recession
                #print('start_ ..... '+gdp.iloc[i-2-j]['year_q']+' ... '+str(gdp.iloc[i-2-j]['GDP']))
                start_ = gdp.iloc[i-1-j]['GDP']
                maybe_ = gdp.iloc[i-1-j-1]['GDP']
                #print('maybe_ ..... '+gdp.iloc[i-2-j-1]['year_q']+' ... '+str(gdp.iloc[i-2-j-1]['GDP']))
                if start_ > maybe_:
                    start_q.append(gdp.iloc[i-1-j+1]['year_q'])
                    break
    
    
    return start_q[0]

In [6]:
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    gdp.columns = ['year_q', 'GDP']
    
    def split_year(row):
        return int(row.split('q')[0]) >= 2000
    
    gdp = gdp[gdp['year_q'].apply(split_year)]       #return only year 2000 above
    
    end_q = []                                      #list of recession ends quarter            
    
    for i in range(2,len(gdp)-2):
        last_2 = gdp.iloc[i-2]['GDP']
        last_ = gdp.iloc[i-1]['GDP']
        current_ = gdp.iloc[i]['GDP']
        next_ = gdp.iloc[i+1]['GDP']
        next_2 = gdp.iloc[i+2]['GDP']
        if last_2 > last_ and current_ < last_ and current_ < next_  and next_ < next_2:      #recession bottom
            end_q.append(gdp.iloc[i+2]['year_q'])
        

    return end_q[0]

In [8]:
get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    gdp.columns = ['year_q', 'GDP']
    
    def split_year(row):
        return int(row.split('q')[0]) >= 2000
    
    gdp = gdp[gdp['year_q'].apply(split_year)]       #return only year 2000 above
    
    bottom_q = []                                    #list of recession bottom quarter            
    
    for i in range(2,len(gdp)-2):
        last_2 = gdp.iloc[i-2]['GDP']
        last_ = gdp.iloc[i-1]['GDP']
        current_ = gdp.iloc[i]['GDP']
        next_ = gdp.iloc[i+1]['GDP']
        next_2 = gdp.iloc[i+2]['GDP']
        if last_2 > last_ and current_ < last_ and current_ < next_  and next_ < next_2:        #recession bottom
            bottom_q.append(gdp.iloc[i]['year_q'])
        
    
    return bottom_q[0]

In [10]:
get_recession_bottom()

'2009q2'

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing_df['State'] = housing_df['State'].map(states)
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    
    housing_df = housing_df[housing_df.columns[4:]].rename(columns=pd.to_datetime)

    sel_col = housing_df.columns[(housing_df.columns >= '2000')]                            #select col range

    housing_df = housing_df[sel_col].resample('Q', axis=1).mean()                           #resample quarter period and apply mean
    housing_df = housing_df.rename(columns=(lambda x: "{}q{}".format(x.year,x.quarter)))    #rename columns

    
    return housing_df

In [12]:
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [13]:
convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:979: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


2010q3        2015q2  2016q4
State RegionName                                    
Ohio  Akron       67566.666667  61833.333333     NaN
      Dayton      57600.000000  50000.000000     NaN

In [14]:
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    the correct ratio is quarter_before_recession_start/recession_bottom
    '''
 
    housing_df = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    
    ##### creates new data showing movement of housing prices
    
    col_list = housing_df.columns.tolist()
    
    housing_df['PriceRatio'] = housing_df[col_list[col_list.index(rec_start)-1]].div(housing_df[rec_bottom])
    housing_df = housing_df[[col_list[col_list.index(rec_start)-1], rec_bottom, 'PriceRatio']]
    
    ##### create 2 groups of data: university and non-university towns
    
    ut_list = ul.to_records(index=False).tolist()
    
    group_1 = housing_df[housing_df.index.isin(ut_list)]                  #housing of university town
    group_2 = housing_df[~housing_df.index.isin(ut_list)]                 #housing of non-university town
     
        
    ##### run a ttest comparing the university town values to the non-university towns values
    
    from scipy import stats
    
    t_value, p = stats.ttest_ind(group_1['PriceRatio'], group_2['PriceRatio'], nan_policy='omit')
    different = p <0.01
    bool(different)
    
    better = None
    if group_1['PriceRatio'].mean() > group_2['PriceRatio'].mean():
        better = "university town"
    else:
        better = "non-university town"

In [15]:
group_1.mean()

2008q2        209552.529183
2009q2        196592.792793
PriceRatio         1.054565
dtype: float64

In [16]:
group_2.mean()

2008q2        242914.466785
2009q2        224128.490677
PriceRatio         1.075196
dtype: float64

In [17]:
different, p, better

(True, 0.002724063704753125, 'non-university town')

In [18]:
stats.ttest_ind(group_2['PriceRatio'],group_1['PriceRatio'], nan_policy='omit')

Ttest_indResult(statistic=2.998032664179183, pvalue=0.002724063704753125)

In [37]:
    housing_df = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    
    ##### creates new data showing movement of housing prices
    
    col_list = housing_df.columns.tolist()
    
    housing_df['PriceRatio'] = housing_df[col_list[col_list.index(rec_start)-1]].div(housing_df[rec_bottom])
    housing_df = housing_df[[col_list[col_list.index(rec_start)-1], rec_bottom, 'PriceRatio']]
    
    ##### create 2 groups of data: university and non-university towns
    
    ut_list = ul.to_records(index=False).tolist()
    
    group_1 = housing_df[housing_df.index.isin(ut_list)]                  #housing of university town
    group_2 = housing_df[~housing_df.index.isin(ut_list)]                 #housing of non-university town
     
        
    ##### run a ttest comparing the university town values to the non-university towns values
    
    from scipy import stats
    
    t_value, p = stats.ttest_ind(group_1['PriceRatio'], group_2['PriceRatio'], nan_policy='omit')
    different = p <0.01
    
    better = None
    if group_1['PriceRatio'].mean() > group_2['PriceRatio'].mean():
        better = "university town"
    else:
        better = "non-university town"

0.9549648035626174

In [39]:
group_2['PriceRatio'].mean()

0.9387558804474

In [42]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    the correct ratio is quarter_before_recession_start/recession_bottom
    '''
 
    housing_df = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    
    ##### creates new data showing movement of housing prices
    
    col_list = housing_df.columns.tolist()
    
    housing_df['PriceRatio'] = housing_df[col_list[col_list.index(rec_start)-1]].div(housing_df[rec_bottom])
    housing_df = housing_df[[col_list[col_list.index(rec_start)-1], rec_bottom, 'PriceRatio']]
    
    ##### create 2 groups of data: university and non-university towns
    
    ut_list = ul.to_records(index=False).tolist()
    
    group_1 = housing_df[housing_df.index.isin(ut_list)]                  #housing of university town
    group_2 = housing_df[~housing_df.index.isin(ut_list)]                 #housing of non-university town
     
        
    ##### run a ttest comparing the university town values to the non-university towns values
    
    from scipy import stats
    
    t_value, p = stats.ttest_ind(group_1['PriceRatio'], group_2['PriceRatio'], nan_policy='omit')
    different = p <0.01
    bool(different)
    
    better = None
    if group_1['PriceRatio'].mean() > group_2['PriceRatio'].mean():
        better = "non-university town"
    else:
        better = "university town"
    
    
    return (different, p, better)

In [43]:
run_ttest()

(True, 0.002724063704753125, 'university town')

In [22]:
def test_q6():
    q6 = run_ttest()
    different, p, better = q6

    res = 'Type test: '
    res += ['Failed\n','Passed\n'][type(q6) == tuple]

    res += 'Test "different" type: '
    res += ['Failed\n','Passed\n'][type(different) == bool or type(different) == np.bool_]

    res += 'Test "p" type: '
    res += ['Failed\n','Passed\n'][type(p) == np.float64]

    res +='Test "better" type: '
    res += ['Failed\n','Passed\n'][type(better) == str]
    if type(better) != str:
        res +='"better" should be a string with value "university town" or  "non-university town"'
        return res
    res += 'Test "different" spelling: '
    res += ['Failed\n','Passed\n'][better in ["university town", "non-university town"]]
    return res
print(test_q6())

Type test: Passed
Test "different" type: Passed
Test "p" type: Passed
Test "better" type: Passed
Test "different" spelling: Passed

